In [ ]:
import numpy as np
import tensorflow as tf
import os
import sys
from sklearn.preprocessing import StandardScaler 

In [ ]:
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils import load_embeddings_and_ids, concatenate_featmats
from Networks import CuratorNet_Precomputation

In [ ]:
resnet50 = load_embeddings_and_ids('../data/resnet_embeddings.npy')

In [ ]:
embedding_list = [
    resnet50,
]

In [ ]:
artwork_ids_set = set()
for embedding in embedding_list:
    if len(artwork_ids_set) == 0:        
        artwork_ids_set.update(embedding['index2id'].values())
    else:
        artwork_ids_set.intersection_update(embedding['index2id'])
artwork_ids = list(artwork_ids_set)
artwork_id2index = {_id:i for i,_id in enumerate(artwork_ids)}
n_artworks = len(artwork_ids)
n_artworks

In [ ]:
featmat_list = [tmp['featmat'] for tmp in embedding_list]
id2index_list = [tmp['id2index'] for tmp in embedding_list]
concat_featmat = concatenate_featmats(artwork_ids, featmat_list, id2index_list)

In [ ]:
concat_featmat = StandardScaler().fit_transform(concat_featmat)

In [ ]:
concat_featmat.shape

In [ ]:
!ls -a /home/mcartagena/Image-Embedding-Learning-and-Feature-Extraction/curatornet_resnet_10m/

In [ ]:
MODEL_PATH = './curatornet_10m/'
MODEL_PATH

In [ ]:
with tf.Graph().as_default():
    network = CuratorNet_Precomputation(
        pretrained_embedding_dim=concat_featmat.shape[1],
        item_layer_units=[200,200],
    )
    gpu_options = tf.GPUOptions(
            per_process_gpu_memory_fraction=0.99,
            allow_growth=True
        )
    config = tf.ConfigProto(gpu_options=gpu_options)
    with tf.Session(config=config) as sess:
        saver = tf.train.Saver()
        saver.restore(sess, tf.train.latest_checkpoint(MODEL_PATH))
        item_vectors = network.precompute_tensors(sess, concat_featmat)

In [ ]:
item_vectors.shape

In [ ]:
item_vectors.dump(MODEL_PATH + "item_vectors.npy")
with open(MODEL_PATH + 'ids', 'w') as f:
    for _id in artwork_ids:
        f.write('%s\n' % _id)